## To Calculate and Save Spectrograms

    Authors: Fernanda Ferreira and Victor Paganotto
    Professor: Tiago F. Tavares
    
    date: Mar/2019

## Import Libraries

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import glob
import librosa
import librosa.display as ld
# Printa todos os valores do array
#np.set_printoptions(threshold=np.inf)
np.set_printoptions(threshold=10)
import pandas as pd
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
import keras
import os
import re
from sklearn.externals import joblib

/home/fernanda/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## To load records

In [2]:
data = glob.glob('1seg/fold1/*.mp3')
#data = glob.glob('../../Projeto_phd/academia/*.wav')
print (len(data))

#19830/5 = 3966
#datas = ['data[0:3965]', 'data[3966:7931]', 'data[7932:11897]', 'data[11898:15863]', 'data[15864:19830]']

19824


In [3]:
len_audios = []
audios = []

for file in data:
    audio, sr = librosa.core.load(file, sr=None, mono=True, offset=0.0, duration=None)
    len_audios.append(len(audio))
    audios.append(audio)
    max_audio = max(len_audios)

hop_length = 400
n_fft = 500

#To convert the hop length and frame size to units of seconds:
#print ("hop length[s]:", float(hop_length)/sr) # units of seconds
#print ("frame size[s]:",float(n_fft)/sr) # units of seconds

In [4]:
#audios dividido por 2:
audios1 = audios[0:9912]
audios2 = audios[9912: 19824]

In [5]:
#audios_resize = []

def my_specs(audios):
    specs = []

    for i in range(len(audios)):
        shape = (max(len_audios))
        i_audio = np.array(audios[i])
        i_audio.resize(shape)
        x = librosa.stft(i_audio, n_fft=n_fft, hop_length=hop_length)
        X = librosa.amplitude_to_db(np.abs(x), ref=np.max)
        specs.append(X)
        #audios_resize.append(i_audio)
    return specs

#Librosa.stft return:
#np.ndarray [shape=(1 + n_fft/2, t), dtype=dtype]

In [6]:
specs1 = my_specs(audios1)

In [7]:
specs2 = my_specs(audios2)

In [8]:
specs_all = specs1 + specs2

In [9]:
specs1_np = np.array(specs1)


In [10]:
specs2_np = np.array(specs2)

In [11]:
specs_all_np = np.concatenate((specs1_np, specs2_np), axis=0)

## To load Labels

In [12]:
l = []
for file in data:
    (dirname, sname) = os.path.split(file)
    (name, ext) = os.path.splitext(sname)
    n = re.split('-', name, flags=re.IGNORECASE)
    n2 = re.split('[0-9]+', n[0])
    #print (n[0])]
    #print (n2[1])
    l.append(n2[1])

In [13]:
def count_elements(seq) -> dict:
    """Tally elements from `seq`."""
    hist = {}
    for i in seq:
        hist[i] = hist.get(i, 0) + 1
    return hist

l_count = count_elements(l)
l_count

{'P': 9980, 'NP': 9844}

## To separate 80% to train and 20% to test

In [14]:
#stratify: certifica a mesma quantidade de audios de cada classe.
x_train1, x_test, y_train1, y_test = train_test_split(specs_all_np, l, test_size=0.5, random_state=42, stratify=l)
x_train, x_val, y_train, y_val = train_test_split(x_train1, y_train1, test_size=0.2, random_state=42, stratify=y_train1)

#print (type(x_train))

## Transform labels in binary

In [15]:
# convert class vectors to binary class matrices
def string_to_binary(lista):
    
    for i in range(len(lista)):
        if lista[i] == 'P':
            lista[i] = 0
        elif lista[i] == 'NP':
            lista[i] = 1
    return lista

In [16]:
y_train_bi = string_to_binary(y_train)
#print (y_train_bi)

y_test_bi = string_to_binary(y_test)
#print (y_test_bi)

y_val_bi = string_to_binary(y_val)
#print (y_val_bi)

print (len(x_val))
print (len(x_train))
print (len(x_test))
#print (x_train_np)

1983
7929
9912


In [19]:
joblib.dump(x_train, "features_propaganda_50_50/1segs/x_train.pkl")
joblib.dump(x_test, "features_propaganda_50_50/1segs/x_test.pkl")
joblib.dump(x_val, "features_propaganda_50_50/1segs/x_val.pkl")
joblib.dump(y_train, "features_propaganda_50_50/1segs/y_train.pkl")
joblib.dump(y_test, "features_propaganda_50_50/1segs/y_test.pkl")
joblib.dump(y_val, "features_propaganda_50_50/1segs/y_val.pkl")


['features_propaganda_50_50/1segs/y_val.pkl']